In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../data/beef/global-meat-production-by-livestock-type.csv')
df.head()

,Entity,Code,Year,Livestock Primary - Sheep and Goat Meat - 1807 - Production - 5510 - tonnes,Livestock Primary - Beef and Buffalo Meat - 1806 - Production - 5510 - tonnes,"Livestock Primary - Meat, pig - 1035 - Production - 5510 - tonnes","Livestock Primary - Meat, game - 1163 - Production - 5510 - tonnes","Livestock Primary - Meat, duck - 1069 - Production - 5510 - tonnes","Livestock Primary - Meat, Poultry - 1808 - Production - 5510 - tonnes","Livestock Primary - Meat, horse - 1097 - Production - 5510 - tonnes","Livestock Primary - Meat, camel - 1127 - Production - 5510 - tonnes","Livestock Primary - Meat, goose and guinea fowl - 1073 - Production - 5510 - tonnes"
0,Afghanistan,AFG,1961,73220.0,43000.0,NaN,4000.0,NaN,5600.0,NaN,3600.0,NaN
1,Afghanistan,AFG,1962,72375.0,45800.0,NaN,4000.0,NaN,6000.0,NaN,4031.0,NaN
2,Afghanistan,AFG,1963,75530.0,47250.0,NaN,5000.0,NaN,6160.0,NaN,5031.0,NaN
3,Afghanistan,AFG,1964,79750.0,48000.0,NaN,5000.0,NaN,6400.0,NaN,4680.0,NaN
4,Afghanistan,AFG,1965,84375.0,48700.0,NaN,6000.0,NaN,6800.0,NaN,4320.0,NaN


In [3]:
df_total = df.groupby('Year').agg(
    Sheep = ('Livestock Primary - Sheep and Goat Meat - 1807 - Production - 5510 - tonnes', 'sum'),
    Beef = ('Livestock Primary - Beef and Buffalo Meat - 1806 - Production - 5510 - tonnes', 'sum'),
    Pig = ('Livestock Primary - Meat, pig - 1035 - Production - 5510 - tonnes', 'sum'),
    Game = ('Livestock Primary - Meat, game - 1163 - Production - 5510 - tonnes' ,'sum'),
    Duck = ('Livestock Primary - Meat, duck - 1069 - Production - 5510 - tonnes','sum'),
    Poultry = ('Livestock Primary - Meat, Poultry - 1808 - Production - 5510 - tonnes' ,'sum'),
    Horse = ('Livestock Primary - Meat, horse - 1097 - Production - 5510 - tonnes' ,'sum'),
    Camel = ('Livestock Primary - Meat, camel - 1127 - Production - 5510 - tonnes' ,'sum'),
    Goose = ('Livestock Primary - Meat, goose and guinea fowl - 1073 - Production - 5510 - tonnes' ,'sum'),
    
).reset_index().rename(
    columns = {'Beef': 'Beef (beef herd)',
               'Pig': 'Pig Meat',
               'Poultry': 'Poultry Meat',
               'Sheep': 'Lamb & Mutton'  ## is this legit??
               
              }
)

In [4]:
annual_production = pd.melt(df_total,
        id_vars=['Year'], 
        value_vars=list(df_total.columns[1:]),
        var_name='Entity', 
        value_name='Global_Production_Tonnes_2018')

annual_production

,Year,Entity,Global_Production_Tonnes_2018
0,1961,Lamb & Mutton,27758362.0
1,1962,Lamb & Mutton,28384749.0
2,1963,Lamb & Mutton,28375386.0
3,1964,Lamb & Mutton,28455867.0
4,1965,Lamb & Mutton,28754686.0
...,...,...,...
517,2014,Goose,10553059.0
518,2015,Goose,10335133.0
519,2016,Goose,10661699.0
520,2017,Goose,11012277.0


# Join to GHG per kg

In [5]:
ghg = pd.read_csv('../data/beef/greenhouse-gas-equivalents/ghg-per-kg-poore.csv')
ghg.head()

,Entity,Code,Year,"GHG emissions per kilogram (Poore & Nemecek, 2018)"
0,Apples,NaN,2010,0.43
1,Bananas,NaN,2010,0.86
2,Barley,NaN,2010,1.18
3,Beef (beef herd),NaN,2010,99.48
4,Beef (dairy herd),NaN,2010,33.30


In [7]:
total_emissions = annual_production.merge(ghg[['Entity', 'GHG emissions per kilogram (Poore & Nemecek, 2018)']], how = 'inner', on = ['Entity'])

# apparently 1 ton is 907.185 kg
total_emissions['total_emissions'] = total_emissions.Global_Production_Tonnes_2018 * 907.185 * total_emissions['GHG emissions per kilogram (Poore & Nemecek, 2018)']
df = total_emissions[['Entity' , 'Year', 'Global_Production_Tonnes_2018', 'GHG emissions per kilogram (Poore & Nemecek, 2018)', 'total_emissions']]

In [8]:
df

,Entity,Year,Global_Production_Tonnes_2018,"GHG emissions per kilogram (Poore & Nemecek, 2018)",total_emissions
0,Lamb & Mutton,1961,27758362.0,39.72,1.000228e+12
1,Lamb & Mutton,1962,28384749.0,39.72,1.022799e+12
2,Lamb & Mutton,1963,28375386.0,39.72,1.022461e+12
3,Lamb & Mutton,1964,28455867.0,39.72,1.025361e+12
4,Lamb & Mutton,1965,28754686.0,39.72,1.036129e+12
...,...,...,...,...,...
227,Poultry Meat,2014,486617249.0,9.87,4.357130e+12
228,Poultry Meat,2015,500868565.0,9.87,4.484735e+12
229,Poultry Meat,2016,515404253.0,9.87,4.614886e+12
230,Poultry Meat,2017,530214964.0,9.87,4.747500e+12


In [9]:
df[df.Year == 2010]

,Entity,Year,Global_Production_Tonnes_2018,"GHG emissions per kilogram (Poore & Nemecek, 2018)",total_emissions
49,Lamb & Mutton,2010,65788254.0,39.72,2.370574e+12
107,Beef (beef herd),2010,298463289.0,99.48,2.693535e+13
165,Pig Meat,2010,468661032.0,12.31,5.233747e+12
223,Poultry Meat,2010,426464848.0,9.87,3.818530e+12


In [10]:
df.to_csv('../data/beef/processed/global_impact_meats.csv')

### Desired Shape: Entity	Code	Year	GHG emissions per 1000kcal (Poore & Nemecek, 2018)